In [ ]:
import os
import sys
import json
import torch
from transformers import AutoTokenizer
import torch.distributed as dist

# 添加项目根目录到路径
sys.path.append(os.path.dirname(os.getcwd()))

from lit.configs.train_config import train_config
from lit.utils.dataset_utils import (
    LatentQADataset, 
    DataCollatorForLatentQA,
    NUM_READ_TOKENS_TO_SHIFT,
    NUM_WRITE_TOKENS_TO_SHIFT,
    DECODER_CHAT_TEMPLATES,
    get_dataset,
    get_dataloaders,
    get_dist_batch_sampler,
    mask_inputs
)
from lit.utils.infra_utils import get_tokenizer, get_model_config_name

In [2]:
args = train_config()

args.batch_size_training = 2
args.train_system = "../data/train/system.json"
args.train_stimulus_completion = "../data/train/stimulus_completion.json"
# args.train_stimulus = "../data/train/stimulus.json"
# args.train_control = "../data/train/control.json"
args.train_qa = "../data/train/qa.json" 
# args.filter = "goal"

model_name = "/data1/ckx/hf-checkpoints/meta-llama/Llama-3.1-8B-Instruct"
    
tokenizer = get_tokenizer(model_name)

# get_dataset函数

In [8]:
dataset = get_dataset(args, tokenizer)

In [ ]:
import json
import random
from collections import defaultdict
train = True
FILTER = args.filter.split("-")
with open(args.train_qa , "r") as f:
    qa_data = json.load(f)

    NUM_QA = max([len(qa_data[label]) for label in qa_data])
    assert NUM_QA == min([len(qa_data[label]) for label in qa_data])

def build_data_and_idx(path):
        # Get data
        print("Loading data from:", path)
        data = defaultdict(list)
        if path == "":
            return data, []
        with open(path, "r") as f:
            raw_data = json.load(f)
            for item in raw_data:
                
                if item["label"].split("-")[0] in FILTER:
                    continue
                data[item["label"]].append(
                    (
                        item.get("system", ""),
                        item.get("control_user", ""),
                        item.get("control_thought", ""),
                        item.get("control_model", ""),
                        item.get("stimulus_user", ""),
                        item.get("stimulus_thought", ""),
                        item.get("stimulus_model", ""),
                    )
                )
        print("Loaded {} labels".format(len(data)))
        # Get id tuples
        NUM_BEHAVIORS = max([len(data[label]) for label in data])
        assert NUM_BEHAVIORS == min([len(data[label]) for label in data])
        print("NUM_BEHAVIORS:", NUM_BEHAVIORS)
        id_tuples = range(len(data) * NUM_BEHAVIORS * NUM_QA)
        print(id_tuples)
        if args.train_percent == 1 or not train:
            id_tuples = list(id_tuples)
        else:
            id_tuples = random.sample(
                id_tuples, int(len(id_tuples) * args.train_percent)
            )
        for i in range(len(id_tuples)):
            label_idx = id_tuples[i] // (NUM_BEHAVIORS * NUM_QA)
            data_idx = (id_tuples[i] // NUM_QA) % NUM_BEHAVIORS
            qa_idx = id_tuples[i] % NUM_QA
            id_tuples[i] = (label_idx, data_idx, qa_idx)
        return data, id_tuples

In [ ]:
FILTER

In [ ]:
qa_data

In [ ]:
p0 = args.train_system if train else args.eval_system
p1 = (
        args.train_stimulus_completion
        if train
        else args.eval_stimulus_completion
    )
p2 = args.train_stimulus if train else args.eval_stimulus
p3 = args.train_control if train else args.eval_control
data_system = build_data_and_idx(p0)
data_stimulus_completion = build_data_and_idx(p1)
data_stimulus = build_data_and_idx(p2)
data_control = build_data_and_idx(p3)

In [ ]:
data_system[0]

In [ ]:
data_system[1]

# dataset类

In [ ]:
dataset.data[1]

In [ ]:
dataset.id_tuples[1]

In [ ]:
dataset.labels[1]

In [ ]:
dataset.qa_data

In [ ]:
dataset.get_behavior_qa(17777)

In [ ]:
dataset.lengths

In [ ]:
len(dataset)

In [ ]:
# dataset[0]
dataset[17777]

In [ ]:
print(dataset[17777]['read_prompt'])

## mask_type = "user"

In [ ]:
print(dataset[17777]['read_prompt'])

In [ ]:
dataset[17777]['dialog']

In [ ]:
dataset[17777]['mask_type']

## mask_type = "system"

In [ ]:
print(dataset[0]["read_prompt"])

In [ ]:
dataset[0]['dialog']

In [ ]:
dataset[0]['mask_type']

# get_dataloaders

In [ ]:
get_dist_batch_sampler(dataset, args, "train")

In [3]:
train_dataloader, eval_dataloader = get_dataloaders(args,tokenizer)

In [4]:
len(train_dataloader)

15688

## lqa_tokenize 函数

In [9]:
batch = [dataset[0]['read_prompt'], dataset[17777]['read_prompt']]
batch

["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nPlease make sure to regularly evaluate the effectiveness of your problem-solving methods and adjust them whenever needed.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nI'm feeling a bit stuck on a challenge I've encountered at work.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n",
 "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nPlease make sure that in your next response, you write a rhetorical question without using any question words.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nCertainly! I'll compose a rhetorical question in my next response without using any question words.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nI've been contemplating the mysteries of the universe lately.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nIs the universe not a vast expanse that defies understanding?<|eot_id|>"]

In [10]:
tokenized_read = tokenizer(batch, 
                   return_tensors="pt",
                   padding=True,
                   add_special_tokens=False,
    )
tokenized_read

{'input_ids': tensor([[128010, 128010, 128010, 128010, 128010, 128010, 128010, 128010, 128010,
         128010, 128010, 128010, 128010, 128010, 128010, 128010, 128010, 128010,
         128010, 128010, 128010, 128010, 128010, 128010, 128010, 128010, 128010,
         128010, 128010, 128010, 128010, 128010, 128010, 128010, 128000, 128006,
           9125, 128007,    271,   5618,   1304,   2771,    311,  15870,  15806,
            279,  27375,    315,    701,   3575,  99246,   5528,    323,   7652,
           1124,  15716,   4460,     13, 128009, 128006,    882, 128007,    271,
             40,   2846,   8430,    264,   2766,  16075,    389,    264,   8815,
            358,   3077,  23926,    520,    990,     13, 128009, 128006,  78191,
         128007,    271],
        [128000, 128006,    882, 128007,    271,   5618,   1304,   2771,    430,
            304,    701,   1828,   2077,     11,    499,   3350,    264,  87068,
           3488,   2085,   1701,    904,   3488,   4339,     13, 1280

In [11]:
print(tokenizer.decode(tokenized_read.input_ids[0]))
print('+++++++++++++++')
print(tokenizer.decode(tokenized_read.input_ids[1]))

<|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Please make sure to regularly evaluate the effectiveness of your problem-solving methods and adjust them whenever needed.<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm feeling a bit stuck on a challenge I've encountered at work.<|eot_id|><|start_header_id|>assistant<|end_header_id|>


+++++++++++++++
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Please make sure that in your next response, you write a rhetorical question without using an

In [12]:
read_lengths = torch.sum(tokenized_read.attention_mask, dim=1)
read_lengths

tensor([49, 83])

In [27]:
mask_inputs._debug_print = True

In [28]:
verb_mask = mask_inputs(tokenized_read.input_ids, 
                        tokenizer.name_or_path,
                        mask_type=['system','user'],
                        # mask_type = None,
                        mask_all_but_last=False)



🔍 MASK DEBUG INFO

📦 Batch 0:
🎯 Mask True positions: [35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58]
🔤 Full sequence with mask:
   <|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Please make sure to regularly evaluate the effectiveness of your problem-solving methods and adjust them whenever needed.<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm feeling a bit stuck on a challenge I've encountered at work.<|eot_id|><|start_header_id|>assistant<|end_h

In [26]:
# 验证字符串编码
encoded = tokenizer.encode("\n\n", add_special_tokens=False)
print(encoded)  # 输出: [271]
for ids in (
        torch.tensor([128006, 9125, 128007, 271]),
        torch.tensor([128006, 882, 128007, 271]),
        torch.tensor([128006, 78191, 128007, 271]),
        torch.tensor([128006, 36013, 128007, 271]),
    ):
    print(tokenizer.decode(ids))

[271]
<|start_header_id|>system<|end_header_id|>


<|start_header_id|>user<|end_header_id|>


<|start_header_id|>assistant<|end_header_id|>


<|start_header_id|>reflect<|end_header_id|>




In [ ]:
verb_mask

In [15]:

verb_lengths = torch.sum(verb_mask, dim=1)
verb_lengths

tensor([24, 48])

In [16]:
read_lengths

tensor([49, 83])

In [17]:
pad_lengths = read_lengths - verb_lengths
pad_lengths

tensor([25, 35])

In [18]:
batch = [dataset[0]['dialog'], dataset[17777]['dialog']]

In [19]:
batch[0]

[{'role': 'assistant', 'content': "Sure, I've analyzed the assistant."},
 {'role': 'user',
  'content': 'What strategy does the assistant use while assisting?'},
 {'role': 'assistant',
  'content': 'The assistant employs a dynamic strategy, frequently evaluating and adapting its problem-solving techniques for better outcomes.'}]

In [20]:
queries = []
for i in range(len(pad_lengths)):
    query = [
        {
                "role": "user",
                "content": "? " * (pad_lengths[i] - 1),
        }
    ]
    query += batch[i]
    print(query)
    queries.append(
            tokenizer.apply_chat_template(
                query,
                tokenize=False,
                add_generation_prompt=False,
                chat_template=(
                    DECODER_CHAT_TEMPLATES[get_model_config_name(model_name)]
                ),
            )
        )



[{'role': 'user', 'content': '? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? '}, {'role': 'assistant', 'content': "Sure, I've analyzed the assistant."}, {'role': 'user', 'content': 'What strategy does the assistant use while assisting?'}, {'role': 'assistant', 'content': 'The assistant employs a dynamic strategy, frequently evaluating and adapting its problem-solving techniques for better outcomes.'}]
[{'role': 'user', 'content': '? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? '}, {'role': 'assistant', 'content': "Sure, I've analyzed the assistant."}, {'role': 'user', 'content': 'If the user shares a speculative thought, what kind of reply should the assistant generate?'}, {'role': 'assistant', 'content': 'The assistant should reply with a rhetorical question that captures the essence of speculation while eschewing typical interrogative language, thus prompting deeper thinking without a direct question format.'}]


In [21]:
queries

["<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?<|eot_id|><|start_header_id|>reflect<|end_header_id|>\n\nSure, I've analyzed the assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat strategy does the assistant use while assisting?<|eot_id|><|start_header_id|>reflect<|end_header_id|>\n\nThe assistant employs a dynamic strategy, frequently evaluating and adapting its problem-solving techniques for better outcomes.<|eot_id|>",
 "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?<|eot_id|><|start_header_id|>reflect<|end_header_id|>\n\nSure, I've analyzed the assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nIf the user shares a speculative thought, what kind of reply should the assistant generate?<|eot_id|><|start_header_id|>reflect<|end_header_id|>\n\nThe assistant should reply with a rhetorical question that captures the

In [22]:
tokenized_write = tokenizer(
        queries,
        return_tensors="pt",
        padding=True,
        add_special_tokens=False,
    )
tokenized_write

{'input_ids': tensor([[128010, 128010, 128010, 128010, 128010, 128010, 128010, 128010, 128010,
         128010, 128010, 128010, 128010, 128010, 128010, 128010, 128010, 128010,
         128010, 128010, 128010, 128010, 128010, 128010, 128010, 128010, 128010,
         128010, 128010, 128010, 128010, 128010, 128000, 128006,    882, 128007,
            271,     30,    949,    949,    949,    949,    949,    949,    949,
            949,    949,    949,    949,    949,    949,    949,    949,    949,
            949,    949,    949,    949,    949,    949,    949, 128009, 128006,
          36013, 128007,    271,  40914,     11,    358,   3077,  30239,    279,
          18328,     13, 128009, 128006,    882, 128007,    271,   3923,   8446,
           1587,    279,  18328,   1005,   1418,  46927,     30, 128009, 128006,
          36013, 128007,    271,    791,  18328,  51242,    264,   8915,   8446,
             11,  14134,  38663,    323,  70817,   1202,   3575,  99246,  12823,
            36

In [23]:
write_lengths = torch.sum(tokenized_write.attention_mask, dim=1)
write_lengths

tensor([ 81, 113])

In [29]:
user_inputs_mask = mask_inputs(
            tokenized_write.input_ids,
            tokenizer.name_or_path,
            mask_type=None,
            shift_start=True,
            modify_chat_template=True,
        )
assert tokenizer.padding_side == "left"



🔍 MASK DEBUG INFO

📦 Batch 0:
🎯 Mask True positions: [32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92]
🔤 Full sequence with mask:
   <|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|python_tag|><|begin_of_text|><|start_header_id|>user<|end_header_id|>

????????????????????????<|eot_id|><|start_header_id|>reflect<|end_header_id|>

Sure, I've analyzed the assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

What strategy does the assistant us

In [25]:
tokenized_write["labels"] = tokenized_write.input_ids.clone()
mask = (tokenized_write.attention_mask == 0) | user_inputs_mask
tokenized_write["labels"][mask] = -100
tokenized_write["labels"]

tensor([[  -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,  40914,     11,    358,   3077,  30239,    279,
          18328,     13, 128009,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,    791,  18328,  51242,    264,   8915,   8446,
             11,  14134,  38663,    323,  70817,   1202,   3575,  99246,  12823,
            369,   2731,  20

# DataCollatorForLatentQA

In [5]:
first_batch = next(iter(train_dataloader))

In [6]:
first_batch

{'tokenized_read': {'input_ids': tensor([[128010, 128010, 128010, 128010, 128010, 128010, 128010, 128010, 128010,
          128010, 128010, 128010, 128010, 128010, 128010, 128010, 128010, 128010,
          128010, 128000, 128006,    882, 128007,    271,   5618,  11142,    279,
           29055,    315,    264,   3674,   9429,  80244,    889,  13919,    430,
            1790,    315,   1148,    584,  45493,    439,   8903,    374,  20968,
            1555,   3674,  11618,    323,  22639,     13,    763,    701,   3938,
           14847,     11,  24564,   1268,   6677,    323,   8830,    527,  27367,
             555,  13042,    323,  13970,  38697,     13, 128009, 128006,  78191,
          128007,    271,  16648,  52784,      0,    358,    690,  27830,    279,
            3560,    315,    264,   3674,   9429,  80244,     11,  21760,    389,
            1268,   1057,  51866,    315,   8903,    527,  20968,   1555,   3674,
           22639,     13,    763,    856,  14847,     11,    358, 